In [1]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.infogram import H2OInfogram
from h2o.estimators.gbm import H2OGradientBoostingEstimator
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import EvaluationModule
pd.options.mode.chained_assignment = None

(CVXPY) Jul 18 06:01:43 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 18 06:01:43 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 18 06:01:43 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
dataset = CoronnaCERTAINDataset(
    library_root='/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge="classification", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='SC', #option: KVB, SC
    imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='bionaive TNF', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    remove_low_DAS = True,
    save_csv=False,
    random_state=2022)

In [4]:
train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()

In [5]:
train.head(10)

,grp,init_group,age,gender,final_education,race_grp,ethnicity,weight,BMI,height,...,md_global_assess,pt_global_assess,di,pt_pain,usresultsCRP,usresultsIgA,usresultsIgG,usresultsIgM,DAS28_CRP_0M,DrugResponse
0,1,0,61.0,0,0,7,1,240.0,39.933728,65.0,...,55.0,50.0,0.250000,60.0,1.000000,44.0,948.0,51.0,3.719481,2
1,1,0,64.0,1,0,7,1,294.0,41.000198,71.0,...,55.0,55.0,0.285714,70.0,16.740000,178.0,882.0,87.0,5.146565,0
2,2,0,51.0,0,2,7,1,90.0,18.175812,59.0,...,65.0,20.0,0.250000,65.0,0.250000,155.0,824.0,43.0,4.425214,0
3,3,0,57.0,0,0,7,1,161.0,30.417361,61.0,...,70.0,55.0,1.000000,65.0,10.637424,273.0,714.0,138.0,5.133521,0
4,2,0,62.0,0,0,7,1,150.0,26.568405,63.0,...,50.0,25.0,0.000000,20.0,17.930000,171.0,992.0,95.0,5.278515,0
5,1,0,54.0,1,2,7,1,182.0,25.381075,71.0,...,40.0,10.0,0.142857,5.0,4.700000,267.0,1110.0,109.0,3.604865,0
6,1,0,50.0,0,0,7,1,250.0,39.151259,67.0,...,35.0,27.0,0.500000,20.0,1.330000,402.0,1290.0,89.0,3.809990,2
7,1,0,37.0,0,0,7,1,198.0,32.945325,65.0,...,44.0,85.0,1.500000,80.0,11.780000,200.0,917.0,180.0,5.772203,1
8,1,0,73.0,0,2,7,1,204.0,37.308012,62.0,...,40.0,35.0,1.125000,25.0,22.060000,238.0,562.0,34.0,4.112486,1
9,1,0,60.0,0,0,7,1,184.0,29.695133,66.0,...,45.0,65.0,0.500000,60.0,1.500000,259.0,981.0,54.0,4.743032,0


In [6]:
# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
train_h2o = h2o.upload_file(str(train_loc))
test_h2o = h2o.upload_file(str(test_loc))

# Identify predictors and response
x = train_h2o.columns[:-1]
# y = "DAS28_CRP_3M"
y = "DrugResponse_binary"

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "13.0.2" 2020-01-14; Java(TM) SE Runtime Environment (build 13.0.2+8); Java HotSpot(TM) 64-Bit Server VM (build 13.0.2+8, mixed mode, sharing)
  Starting server from /Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmposzpw6dy
  JVM stdout: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmposzpw6dy/h2o_gaskell_started_from_python.out
  JVM stderr: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmposzpw6dy/h2o_gaskell_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,12 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 22 days
H2O_cluster_name:,H2O_from_python_gaskell_6nq8dn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
train_h2o

grp,init_group,age,gender,final_education,race_grp,ethnicity,weight,BMI,height,newsmoker,drinker,drinksperwk,duration_ra,ara_func_class,num_tnf,num_nontnf,ndmardused,rfstatus_b,usresultsRF,ccpstatus_b,usresultsCCP3,hxmi,hxunstab_ang,hxchf,hxother_cv,hxtia,hxstroke,hxcopd,hxhtn,hxdiabetes,hxhld,hx_anycancer,seatedbp1,seatedbp2,pres_mtx,pres_arava,pres_azulfidine,pres_plaquenil,pres_imuran,pres_minocin,pres_pred,nonpresNSAIDs_use,NSAIDs_use,tender_jts_28,swollen_jts_28,md_global_assess,pt_global_assess,di,pt_pain,usresultsCRP,usresultsIgA,usresultsIgG,usresultsIgM,DAS28_CRP_0M,DrugResponse
1,0,61,0,0,7,1,240,39.9337,65,2,2,0.466667,1,0,0,0,1,0,9.8,0,15,0,0,0,0,0,0,0,0,1,0,0,110,80,1,0,0,0,0,0,1,0,0,3,9,55,50,0.25,60,1,44,948,51,3.71948,2
1,0,64,1,0,7,1,294,41.0002,71,2,2,2,2,1,0,0,1,0,8.7,0,15,0,0,0,0,0,0,0,0,0,0,0,137,87,1,0,0,0,0,0,0,0,1,6,13,55,55,0.285714,70,16.74,178,882,87,5.14656,0
2,0,51,0,2,7,1,90,18.1758,59,0,1,0,5,1,0,0,1,1,30.5,1,174,0,0,0,0,0,0,0,0,0,0,0,100,70,1,0,0,0,0,0,1,0,0,14,13,65,20,0.25,65,0.25,155,824,43,4.42521,0
3,0,57,0,0,7,1,161,30.4174,61,3,1,0,3,1,0,0,1,1,110.2,1,251,0,0,0,0,0,0,0,1,0,1,0,132,79,1,0,0,0,0,0,1,0,0,9,9,70,55,1,65,10.6374,273,714,138,5.13352,0
2,0,62,0,0,7,1,150,26.5684,63,2,2,14,2,1,0,0,1,0,9.4,0,15,0,0,0,0,0,0,0,1,0,0,0,112,75,1,0,0,0,0,0,0,0,0,12,12,50,25,0,20,17.93,171,992,95,5.27851,0
1,0,54,1,2,7,1,182,25.3811,71,0,2,0.466667,2,0,0,0,1,1,750,1,251,0,0,0,0,0,0,0,0,0,0,0,119,76,1,0,0,0,0,0,0,0,0,5,5,40,10,0.142857,5,4.7,267,1110,109,3.60486,0
1,0,50,0,0,7,1,250,39.1513,67,2,2,0.466667,0,1,0,0,1,1,92.4,1,73,0,0,0,0,0,0,0,0,0,0,0,110,80,0,0,0,0,0,0,0,0,0,7,6,35,27,0.5,20,1.33,402,1290,89,3.80999,2
1,0,37,0,0,7,1,198,32.9453,65,0,2,0.466667,0,1,0,0,0,0,8.5,0,15,0,0,0,0,0,0,0,0,0,0,0,137,81,0,0,0,0,0,0,1,1,0,13,6,44,85,1.5,80,11.78,200,917,180,5.7722,1
1,0,73,0,2,7,1,204,37.308,62,3,1,0,4,1,0,0,2,1,259.3,1,188,0,0,0,0,0,0,0,1,1,0,0,177,100,0,0,1,1,0,0,1,1,0,2,7,40,35,1.125,25,22.06,238,562,34,4.11249,1
1,0,60,0,0,7,1,184,29.6951,66,3,2,0.466667,8,1,0,0,2,1,61,0,15,0,0,0,0,0,0,0,0,0,1,0,141,100,1,0,0,0,0,0,0,1,1,11,6,45,65,0.5,60,1.5,259,981,54,4.74303,0


In [8]:
for feature in dataset.categorical:
    train_h2o[feature] = train_h2o[feature].asfactor()
    test_h2o[feature] = test_h2o[feature].asfactor()
train_h2o[y] = train_h2o[y].asfactor()
test_h2o[y] = test_h2o[y].asfactor()

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Column DrugResponse_binary not found
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_10_sid_a89f (cols_py (tmp= py_9_sid_a89f (:= (tmp= py_8_sid_a89f (:= (tmp= py_7_sid_a89f (:= (tmp= py_6_sid_a89f (:= (tmp= py_5_sid_a89f (:= (tmp= py_4_sid_a89f (:= (tmp= py_3_sid_a89f (:= (tmp= py_2_sid_a89f (:= (tmp= py_1_sid_a89f (:= Key_Frame__upload_a5e75d1b3c38ceaa6f897f2111674369.hex (as.factor (cols_py Key_Frame__upload_a5e75d1b3c38ceaa6f897f2111674369.hex 'grp')) 0 [])) (as.factor (cols_py py_1_sid_a89f 'init_group')) 1 [])) (as.factor (cols_py py_2_sid_a89f 'gender')) 3 [])) (as.factor (cols_py py_3_sid_a89f 'final_education')) 4 [])) (as.factor (cols_py py_4_sid_a89f 'race_grp')) 5 [])) (as.factor (cols_py py_5_sid_a89f 'ethnicity')) 6 [])) (as.factor (cols_py py_6_sid_a89f 'newsmoker')) 10 [])) (as.factor (cols_py py_7_sid_a89f 'drinker')) 11 [])) (as.factor (cols_py py_8_sid_a89f 'ara_func_class')) 14 [])) '

In [ ]:
# Run AutoML for 20 base models
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "SC_classification_Jul9")
aml.train(x=x, y=y, training_frame=train_h2o, leaderboard_frame = test_h2o)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
lb['model_id']

In [ ]:
# aml_sc = EvaluationModule.AutoBuild(seed=1, project_name="EHR_RA_SC", challenge=dataset.challenge)
# test, test_loc = dataset.get_test()

# for model in lb.as_data_frame()['model_id']:
#     m = h2o.get_model(model)
#     pred = m.predict(test_h2o).as_data_frame()
#     aml_sc.evaluate(model, test, pred)

In [ ]:
# aml_sc = evaluate.AutoBuild(seed=1, project_name="EHR_RA_SC")
# test, test_loc = dataset.get_test()
# baseline = test['DAS28_CRP_0M']
# true = test['DAS28_CRP_3M']

In [ ]:
# pred = pred.as_data_frame()

In [ ]:
# aml_sc.evaluate("stackedensemble", test, pred)

In [ ]:
# regression, classification = aml_sc.leaderboard()

In [ ]:
# regression

In [ ]:
# classification.sort_values('Accuracy',ascending=False)

In [ ]:
gbm = H2OGradientBoostingEstimator(nfolds = 5, seed = 1)
gbm.train(x=x, y=y, training_frame=train_h2o)

In [ ]:
gbm.auc(xval=True)

In [ ]:
perf = aml.leader.model_performance(test_h2o)
perf

In [ ]:
# Explain an AutoML object
exa = aml.explain(test_h2o)